In [ ]:
!unzip /content/drive/MyDrive/MLProject/genius_lyrics.zip -d .

Archive:  /content/drive/MyDrive/MLProject/genius_lyrics.zip
  inflating: ./song_lyrics.csv       


In [ ]:
import pandas as pd
file_path = 'song_lyrics.csv'

In [ ]:
chunksize = 10000  # Adjust this depending on the size of your file and available memory
filtered_data = []

for chunk in pd.read_csv(file_path, chunksize=chunksize, usecols=['id', 'lyrics', 'language_ft','title','artist']):
    filtered_chunk = chunk[chunk['language_ft'] == 'en']
    filtered_data.append(filtered_chunk)

final_df = pd.concat(filtered_data, ignore_index=True)

In [ ]:
final_df[final_df['id']==3278026]['artist']

1602440    M
Name: artist, dtype: object

In [ ]:
print (final_df.head())

               title  tag     artist  id language_ft
0          Killa Cam  rap    Cam'ron   1          en
1         Can I Live  rap      JAY-Z   3          en
2  Forgive Me Father  rap   Fabolous   4          en
3       Down and Out  rap    Cam'ron   5          en
4             Fly In  rap  Lil Wayne   6          en


In [ ]:
final_df.dropna(subset=['title','artist','id'], inplace=True)

In [ ]:
import re
def preprocess_column(column):
    column = column.str.replace(r"\(.*\)", "", regex=True)
    column = column.str.replace(r"\[.*\]", "", regex=True)
    column = column.str.replace(r"\{.*\}", "", regex=True)
    column = column.str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)
    column = column.str.replace(r"\s+", "", regex=True)
    column = column.str.lower()
    column = column.str.replace("remix", "")
    column = column.str.replace("feat", "")
    column = column.str.replace("ft.", "")
    column = column.str.replace("ft\.", "")
    column = column.str.replace("ft", "")
    return column

In [ ]:
tracks_df = pd.read_csv('output3.csv')

In [ ]:
print (tracks_df.head())

      video_id                                          file_name  \
0  -1K1j3tUt6g         -1K1j3tUt6g#rock#Storywriter-Supercar.webm   
1  -2jkPmVwow4  -2jkPmVwow4#rock#Aya - Rojou No Kage (Shadows ...   
2  -6MbrH_fbT8  -6MbrH_fbT8#rock#Andra And The Backbone - Hita...   
3  -i2Kn-EXh_A  -i2Kn-EXh_A#rock#Super Furry Animals - (Drawin...   
4  -jidITdmeNM        -jidITdmeNM#rock#Nell (넬) - Good Night.webm   

                              recording_ids                  artist  \
0  ['1d2db7a9-4638-4bf2-a960-fd098cb25582']                SUPERCAR   
1  ['36ff749e-a832-48fb-9d29-d575820055d6']                      亜矢   
2  ['30c4088d-483b-4df1-b85e-22ec4e464700']  Andra and the BackBone   
3  ['d0ee5f7b-cf80-4f2d-8170-f288450cdd77']     Super Furry Animals   
4  ['e95fa370-6ad0-437c-9d3a-0c097cc1a6e6']                    Nell   

                              title    genres      processed_title  \
0                       STORYWRITER  ['rock']          storywriter   
1                 

In [ ]:
for col in ['title', 'artist']:
    final_df[f'processed_{col}'] = preprocess_column(final_df[col])
    tracks_df[f'processed_{col}'] = preprocess_column(tracks_df[col])

In [ ]:
print (final_df.tail())
print (final_df.shape)

                             title      tag                    artist  \
3415309  Everything Is Alright Now      pop             Chuck Bernard   
3415310                 White Lies      pop                  ElementD   
3415311                      Ocean      pop                   Effemar   
3415312            Raise Our Hands      pop  Culture Code, Pag & Mylo   
3415313                 New Number  country         Alana Springsteen   

              id language_ft         processed_title    processed_artist  
3415309  7882838          en  everythingisalrightnow        chuckbernard  
3415310  7882840          en               whitelies            elementd  
3415311  7882842          en                   ocean             effemar  
3415312  7882845          en           raiseourhands  culturecodepagmylo  
3415313  7882848          en               newnumber    alanaspringsteen  
(3415205, 7)


In [ ]:
print (tracks_df.head())
print (tracks_df.shape)

      video_id                                          file_name  \
0  -1K1j3tUt6g         -1K1j3tUt6g#rock#Storywriter-Supercar.webm   
1  -2jkPmVwow4  -2jkPmVwow4#rock#Aya - Rojou No Kage (Shadows ...   
2  -6MbrH_fbT8  -6MbrH_fbT8#rock#Andra And The Backbone - Hita...   
3  -i2Kn-EXh_A  -i2Kn-EXh_A#rock#Super Furry Animals - (Drawin...   
4  -jidITdmeNM        -jidITdmeNM#rock#Nell (넬) - Good Night.webm   

                              recording_ids                  artist  \
0  ['1d2db7a9-4638-4bf2-a960-fd098cb25582']                SUPERCAR   
1  ['36ff749e-a832-48fb-9d29-d575820055d6']                      亜矢   
2  ['30c4088d-483b-4df1-b85e-22ec4e464700']  Andra and the BackBone   
3  ['d0ee5f7b-cf80-4f2d-8170-f288450cdd77']     Super Furry Animals   
4  ['e95fa370-6ad0-437c-9d3a-0c097cc1a6e6']                    Nell   

                              title    genres      processed_title  \
0                       STORYWRITER  ['rock']          storywriter   
1                 

In [ ]:
merged_df = pd.merge(tracks_df,final_df, left_on='processed_title', right_on='processed_title', how='inner')

In [ ]:
merged_df.columns

Index(['video_id', 'file_name', 'recording_ids', 'artist_x', 'title_x',
       'genres', 'processed_title', 'processed_artist_x', 'title_y', 'tag',
       'artist_y', 'id', 'language_ft', 'processed_artist_y'],
      dtype='object')

In [ ]:
merged_df.shape

(152197, 14)

In [ ]:
mask = (merged_df['processed_artist_x'] == '') | (merged_df['processed_artist_y'] == '') | (merged_df['processed_title'] == '')
merged_df = merged_df[~mask]
print (merged_df.shape)

(140204, 14)


In [ ]:
merged_df = merged_df[merged_df.apply(lambda row: (row['processed_artist_y'] in row['processed_artist_x']), axis=1)]

In [ ]:
print (merged_df.shape)
print (merged_df.columns)

(1880, 14)
Index(['video_id', 'file_name', 'recording_ids', 'artist_x', 'title_x',
       'genres', 'processed_title', 'processed_artist_x', 'title_y', 'tag',
       'artist_y', 'id', 'language_ft', 'processed_artist_y'],
      dtype='object')


In [ ]:
print (merged_df.head())

          video_id                                          file_name  \
11304  -i2Kn-EXh_A  -i2Kn-EXh_A#rock#Super Furry Animals - (Drawin...   
11612  -JJAXwAaA2w  -JJAXwAaA2w#rock#Placebo - The Bitter End (Off...   
11657  -nqRkAsZumc            -nqRkAsZumc#rock#Incubus - Stellar.webm   
11683  -Ue193sAcrE  -Ue193sAcrE#rock#Maximo Park - Apply Some Pres...   
11686  -_EprMTJPWA  -_EprMTJPWA#rock#Idlewild - American English.webm   

                                           recording_ids             artist_x  \
11304           ['d0ee5f7b-cf80-4f2d-8170-f288450cdd77']  Super Furry Animals   
11612  ['3d76b996-e1fd-45e9-bfa4-82938799a9e8', '62ca...              Placebo   
11657  ['0b3773b6-771b-4da8-ab0e-c2810fb57dbb', '10b6...              Incubus   
11683           ['564198ac-e29d-47ea-beca-d862bc126cfd']          Maxïmo Park   
11686  ['471b19b8-78b5-47a1-a407-1f8ea8a27724', '905e...             Idlewild   

                                title_x    genres      processed_title  \


In [ ]:
new_df = merged_df[['title_x','artist_x','id', 'video_id', 'genres', 'tag', 'file_name','recording_ids']].rename(columns={'id': 'genius_id', 'artist_x': 'artist' , 'title_x': 'title' , 'tag': 'genres_genius' , 'genres': 'genres_self'})
print (new_df.shape)

(1880, 8)


In [ ]:
!pip install musicbrainzngs

In [ ]:
import requests
import musicbrainzngs as mb
import time
import ast
mb.set_useragent("Fetching-genres-cse512-project", "0.1", "xyz@abc.com")

In [ ]:
def fetch_high_level_data(recording_ids, attempts):
    base_url = "https://acousticbrainz.org/api/v1/high-level"
    params = {'recording_ids': ';'.join(recording_ids)}
    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200:
            res = response.json()
            for r_id in recording_ids:
                data = res.get(r_id, {})
                if data:
                    return data, r_id
            return {}, ''  # Return empty values if no data is found
        else:
            if attempts == 0:
                time.sleep(1)
                return fetch_high_level_data(recording_ids, 1)
            else:
                return {}, ''  # Return empty values if status code is not 200
    except Exception as e:
        if attempts == 0:
            time.sleep(1)
            return fetch_high_level_data(recording_ids, 1)
        else:
            time.sleep(1)
            return {}, ''  # Return empty values in case of an exception


In [ ]:
def get_mood_and_genre(data):
    data = data.get('0', {})
    genre = data.get('metadata', {}).get('tags',{}).get('genre','')
    data = data.get('highlevel', {})
    if not data:
        return [],genre
    mood_scores = {
        "dance": data.get("danceability", {}).get("all", {}).get("danceable",0),
        "aggressive": data.get("mood_aggressive", {}).get("all", {}).get("aggressive", 0),
        "happy": data.get("mood_happy", {}).get("all", {}).get("happy", 0),
        "party": data.get("mood_party", {}).get("all", {}).get("party", 0),
        "relaxed": data.get("mood_relaxed", {}).get("all", {}).get("relaxed", 0),
        "sad": data.get("mood_sad", {}).get("all", {}).get("sad", 0)
    }
    if mood_scores['happy'] > mood_scores['sad']:
        mood_scores.pop('sad', None)
    else:
        mood_scores.pop('happy', None)

    filtered_moods = {mood: score for mood, score in mood_scores.items() if score > 0.5}
    if not filtered_moods:
        return [],genre

    sorted_moods = sorted(filtered_moods, key=filtered_moods.get, reverse=True)
    return sorted_moods[:3],genre

In [ ]:
def get_genre_by_recording_ids(recording_ids):
    for recording_id in recording_ids:
      try:
        result = mb.get_recording_by_id(recording_id, includes=["tags"])
        tags = result.get('recording', {}).get('tag-list',[])
        if tags:
          return sorted([(tag['count'], tag['name']) for tag in tags], reverse=True)
      except Exception as e:
        time.sleep(0.5)
        continue
    return []

In [ ]:
new_df.shape

(1880, 8)

In [ ]:
c=0
selected_mbid_list = []
mood_list = []
genres_mb_list = []
genres_ab_list = []

for index, row in new_df.iterrows():
  try:
    high_level_data = {}
    selected_mbid = ''
    moods = []
    genre_ab = []
    genre_mb = []

    recording_ids = ast.literal_eval(row['recording_ids'])
    high_level_data , selected_mbid = fetch_high_level_data(recording_ids,0)
    moods , genre_ab = get_mood_and_genre(high_level_data)
    genre_mb = get_genre_by_recording_ids(recording_ids)[:3]

    selected_mbid_list.append(selected_mbid)
    mood_list.append(moods)
    genres_ab_list.append(genre_ab)
    genres_mb_list.append(genre_mb)

  except Exception as e:
    print (e)
    selected_mbid_list.append(selected_mbid)
    mood_list.append(moods)
    genres_ab_list.append(genre_ab)
    genres_mb_list.append(genre_mb)
    continue

  c+=1
  if c%25==0:
    print (f"Done: {c*100/1880}%")

Done: 1.3297872340425532%
Done: 2.6595744680851063%
Done: 3.9893617021276597%
Done: 5.319148936170213%
Done: 6.648936170212766%
Done: 7.9787234042553195%
Done: 9.308510638297872%
Done: 10.638297872340425%
Done: 11.96808510638298%
Done: 13.297872340425531%
Done: 14.627659574468085%
Done: 15.957446808510639%
Done: 17.28723404255319%
Done: 18.617021276595743%
Done: 19.9468085106383%
Done: 21.27659574468085%
Done: 22.606382978723403%
Done: 23.93617021276596%
Done: 25.26595744680851%
Done: 26.595744680851062%
Done: 27.925531914893618%
Done: 29.25531914893617%
Done: 30.585106382978722%
Done: 31.914893617021278%
Done: 33.244680851063826%
Done: 34.57446808510638%
Done: 35.90425531914894%
Done: 37.234042553191486%
Done: 38.56382978723404%
Done: 39.8936170212766%
Done: 41.223404255319146%
Done: 42.5531914893617%
Done: 43.88297872340426%
Done: 45.212765957446805%
Done: 46.54255319148936%
Done: 47.87234042553192%
Done: 49.202127659574465%
Done: 50.53191489361702%
Done: 51.861702127659576%
Done: 53

In [ ]:
new_df['selected_mbid'] = selected_mbid_list
new_df['mood'] = mood_list
new_df['genres_mb'] = genres_mb_list
new_df['genres_ab'] = genres_ab_list

In [ ]:
file_path = '/content/drive/MyDrive/new_df2.csv'
new_df.to_csv(file_path, index=False)

In [ ]:
tracks_df = pd.read_csv('/content/drive/MyDrive/MLProject/songs.csv')

In [ ]:
print (final_df.shape)
print (tracks_df.shape)

(3415314, 3)
(1880, 12)


In [ ]:
final_df.dropna(subset=['id'], inplace=True)
tracks_df.dropna(subset=['genius_id'] , inplace=True)

In [ ]:
merged_df = pd.merge(tracks_df , final_df , how="inner" , left_on="genius_id" ,right_on='id')
print (merged_df.shape)
print (merged_df.columns)

(1880, 15)
Index(['title', 'artist', 'genius_id', 'video_id', 'genres_self',
       'genres_genius', 'file_name', 'recording_ids', 'selected_mbid', 'mood',
       'genres_mb', 'genres_ab', 'lyrics', 'id', 'language_ft'],
      dtype='object')


In [ ]:
new_df = merged_df[['title', 'artist', 'genius_id', 'video_id', 'genres_self','genres_genius', 'recording_ids', 'selected_mbid', 'mood', 'genres_mb', 'genres_ab', 'lyrics']]
print (new_df.shape)

(1880, 12)


In [ ]:
new_df.to_csv('/content/drive/MyDrive/MLProject/lyrics.csv', index=False)